In [157]:
#Import libraries

import gzip #for parsing gz files
import pandas as pd
import re 
#import inflect #numeric/cardinal

In [286]:
import warnings
warnings.filterwarnings('ignore')

In [713]:
dirPath = r"\data"
fileName = r"reviews_Tools_and_Home_Improvement_5.json.gz"
fileName = r"reviews_Musical_Instruments_5.json.gz"

path = dirPath + "\\" + fileName

In [714]:
import os
current = os.getcwd()
current
path = current + path
path

'C:\\Study\\Projects\\NLP\\data\\reviews_Musical_Instruments_5.json.gz'

In [692]:
dataFile = gzip.open(path, 'r')

In [693]:
# Need this function because large amount of data we are going to load up

# http://jmcauley.ucsd.edu/data/amazon/index.html
# eval function allows text to be parsed as python code
# parse function is a generator with yield
# yield when we want to iterate over a sequence, but don't want to store the entire sequence in memory
def parse(dataFile):
    for each in dataFile:
        yield eval(each)

In [694]:
# for each review for a product in dataFile maps dict[nth] = review (nth)
# enumerate can't be used on a generator
count = 0
productReviews = {}
for eachReview in parse(dataFile):
    productReviews[count] = eachReview
    count = count + 1


In [695]:
productReviews[2]

{'reviewerID': 'A195EZSQDW3E21',
 'asin': '1384719342',
 'reviewerName': 'Rick Bennette "Rick Bennette"',
 'helpful': [1, 1],
 'reviewText': 'The primary job of this device is to block the breath that would otherwise produce a popping sound, while allowing your voice to pass through with no noticeable reduction of volume or high frequencies. The double cloth filter blocks the pops and lets the voice through with no coloration. The metal clamp mount attaches to the mike stand secure enough to keep it attached. The goose neck needs a little coaxing to stay where you put it.',
 'overall': 5.0,
 'summary': 'It Does The Job Well',
 'unixReviewTime': 1377648000,
 'reviewTime': '08 28, 2013'}

In [241]:
len(productReviews)

10261

In [242]:
#productReviews[101718]

In [243]:
productsReviewData = pd.DataFrame.from_dict(productReviews, orient='index')

In [244]:
productsReviewData.shape
productsReviewData.columns
productsReviewData.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]","The product does exactly as it should and is quite affordable.I did not realized it was double screened until it arrived, so it was even better than I had expected.As an added bonus, one of the screens carries a small hint of the smell of an old grape candy I used to buy, so for reminiscent's sake, I cannot stop putting the pop filter next to my nose and smelling it after recording. :DIf you needed a pop filter, this will work just as well as the expensive ones, and it may even come with a p...",5.0,Jake,1363392000,"03 16, 2013"


In [722]:
# reduce what we really only need > focus on product id / review
reduceData = productsReviewData.loc[:, ['asin', 'reviewText', 'helpful', 'overall']]
reduceData.shape

(10261, 4)

In [723]:
neutralRating = reduceData.loc[reduceData['overall']==3.0, :]
neutralRating.shape

(772, 4)

In [724]:
# The index no longer has the right indexes as we remove rating '3' due to neural 
data = notNeutralRating = reduceData.loc[reduceData['overall']!=3.0, :]
data.shape

(9489, 4)

In [725]:
rateTheReview = lambda x: 1.0 if x > 2.0 else 0.0 #Negative/Positive rating 

In [726]:
# Retain the original overall Rating reviews (1 - 5 )
# now we apply 1 for positive review, 0 for negative review
data['trueRating'] = data['overall'].copy().apply(rateTheReview)

In [731]:
# reset the index 
data = data.reset_index()
data.rename(columns={"index": "masterIndex"}, inplace=True)
data.shape

(9489, 7)

In [732]:
data.shape, data.columns

((9489, 7),
 Index(['level_0', 'masterIndex', 'asin', 'reviewText', 'helpful', 'overall',
        'trueRating'],
       dtype='object'))

In [733]:
data.tail(2)

,level_0,masterIndex,asin,reviewText,helpful,overall,trueRating
9487,9487,10259,B00JBIVXGC,"Well, MADE by Elixir and DEVELOPED with Taylor Guitars ... these strings were designed for the new 800 (Rosewood) series guitars that came out this year (2014) ... the promise is a &#34;bolder high end, fuller low end&#34; ... I am a long-time Taylor owner and favor their 800 series (Rosewood/Spruce is my favorite combo in tone woods) ... I have almost always used Elixir Nanoweb Phosphor Bronze lights on my guitars ... I like not only the tone but the feel and longevity of these strings ... ...","[0, 0]",4.0,1.0
9488,9488,10260,B00JBIVXGC,"These strings are really quite good, but I wouldn't call them perfect. The unwound strings are not quite as bright as I am accustomed to, but they still ring nicely. This is the only complaint I have about these strings. If the unwound strings were a tiny bit brighter, these would be 5-star strings. As it stands, I give them 4.5 stars... not a big knock, actually.The low-end on the wound strings is very nice and quite warm. I put these on a jumbo and it definitely accentuates the &#34;j...","[0, 0]",4.0,1.0


In [734]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
import Contractions_Words as contractions
import importlib
importlib.reload(contractions)

contractions_dict = contractions.contractions_dict

#print(contractions_dict.keys())
#print(contractions_dict["s***"])

pattern = '%s' % '|'.join(contractions_dict.keys())
#pattern = r"%s" % pattern
#print(pattern)
contractions_re = re.compile(pattern)
def expand_contractions(s, contractions_dict=contractions_dict):
    #edit for special case like s*** etc
    s = re.sub("\*", "#", s)
    def replace(match):
        #print(type(match), match)
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)


In [735]:
sample = "I don\'t know what to say nay. I haven\'t. s###"
expand_contractions(sample)

'I do not know what to say nay. I have not. shit'

In [736]:
#from ntlk import tokenize
import nltk
import string

In [737]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer() 

# return both wholeText, paragraph Text
def singleReviewCleanUp(sample):
    #print(sample)
    #sample = sample.decode("utf-8")
    sample = expand_contractions(sample)
    sample = sample.lower()
    
    #for word in sample:
    #    print(lemmatizer.lemmatize(word, wordnet.VERB))
    
    review = sent_tokenize(sample)
    #print(review)
    for count, eachSent in enumerate(review):
        #remove numbers
        eachSent = re.sub("r'\d+/\d+|\d+|\$\d+.\d+", " ", eachSent)
        #remove string
        eachSent = re.sub('[%s]' % re.escape(string.punctuation), ' ', eachSent)
        #word tokens
        #print([lemmatizer.lemmatize(text, wordnet.VERB) for text in eachSent.split()])
        eachSent = word_tokenize(eachSent)
        review[count] = eachSent
        # lemmatize and remove stopWords
        #eachSent = [word for word in eachSent if not word in stopWords]
        eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent]
        eachSent = [lemmatizer.lemmatize(word, wordnet.NOUN) for word in eachSent]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent if not word in stopWords]
        review[count] = " ".join(eachSent)
    wholeText = [" ".join(review) ]
    paragraphText = review
    #return wholeText
    #return paragraphText
    return paragraphText, wholeText

#paragraphText, wholeText = singleReviewCleanUp(sample)
#wholeText, paragraphText
#paragraphText

In [738]:
#Give me back text in a paragraph sentence forms. They do have more values than a clobbed of text based on Polarity Check.
cleanUpTextParagraph = lambda text: singleReviewCleanUp(text)[0]


In [739]:
#%%time
data['Processed'] = data.reviewText.apply(cleanUpTextParagraph)


In [740]:
data['CleanedJoin'] = data['Processed'].apply(', '.join)

In [741]:
data.shape, data.columns

((9489, 9),
 Index(['level_0', 'masterIndex', 'asin', 'reviewText', 'helpful', 'overall',
        'trueRating', 'Processed', 'CleanedJoin'],
       dtype='object'))

In [742]:
data.tail(3)

,level_0,masterIndex,asin,reviewText,helpful,overall,trueRating,Processed,CleanedJoin
9486,9486,10258,B00JBIVXGC,"I have tried coated strings in the past ( including Elixirs) and have never been very fond of them. Whenever I tried them I felt a certain disconnect from my guitar. Somewhat reminiscent of wearing condom. Not that I hated them, just didn't really love them. These are the best ones I've tried so far. I still don't like them as much as regular strings but because of the type of gigs I mostly do these seem to be a reasonable trade off. If you need a longer lasting string for whatever the reaso...","[0, 0]",4.0,1.0,"[i have try coat string in the past include elixir and have never be very fond of them, whenever i try them i felt a certain disconnect from my guitar, somewhat reminiscent of wear condom, not that i hat them just do not really love them, these be the best one i have try so far, i still do not like them a much a regular string but because of the type of gig i mostly do these seem to be a reasonable trade off, if you need a longer last string for whatever the reason these be really the best o...","i have try coat string in the past include elixir and have never be very fond of them, whenever i try them i felt a certain disconnect from my guitar, somewhat reminiscent of wear condom, not that i hat them just do not really love them, these be the best one i have try so far, i still do not like them a much a regular string but because of the type of gig i mostly do these seem to be a reasonable trade off, if you need a longer last string for whatever the reason these be really the best ou..."
9487,9487,10259,B00JBIVXGC,"Well, MADE by Elixir and DEVELOPED with Taylor Guitars ... these strings were designed for the new 800 (Rosewood) series guitars that came out this year (2014) ... the promise is a &#34;bolder high end, fuller low end&#34; ... I am a long-time Taylor owner and favor their 800 series (Rosewood/Spruce is my favorite combo in tone woods) ... I have almost always used Elixir Nanoweb Phosphor Bronze lights on my guitars ... I like not only the tone but the feel and longevity of these strings ... ...","[0, 0]",4.0,1.0,[well make by elixir and develop with taylor guitar these string be design for the new rosewood series guitar that come out this year the promise be a bolder high end fuller low end i be a long time taylor owner and favor their series rosewood spruce be my favorite combo in tone wood i have almost always use elixir nanoweb phosphor bronze light on my guitar i like not only the tone but the feel and longevity of these string i have never have any issue with elixir nanowebs i recently pick up ...,well make by elixir and develop with taylor guitar these string be design for the new rosewood series guitar that come out this year the promise be a bolder high end fuller low end i be a long time taylor owner and favor their series rosewood spruce be my favorite combo in tone wood i have almost always use elixir nanoweb phosphor bronze light on my guitar i like not only the tone but the feel and longevity of these string i have never have any issue with elixir nanowebs i recently pick up a...
9488,9488,10260,B00JBIVXGC,"These strings are really quite good, but I wouldn't call them perfect. The unwound strings are not quite as bright as I am accustomed to, but they still ring nicely. This is the only complaint I have about these strings. If the unwound strings were a tiny bit brighter, these would be 5-star strings. As it stands, I give them 4.5 stars... not a big knock, actually.The low-end on the wound strings is very nice and quite warm. I put these on a jumbo and it definitely accentuates the &#34;j...","[0, 0]",4.0,1.0,"[these string be really quite good but i would not call them perfect, the unwind string be not quite a bright a i be accustom to but they still ring nicely, this be the only complaint i have about these string, if the unwind string be a tiny bite brighter these would be star 

In [719]:
#data.to_json("cleanData_reviews_Musical_Instruments_5.json.gz", orient='index')

data.drop(columns=['Processed']).to_pickle("cleaned"_fileName.split('.')[0] + "pkl.gz")

In [302]:
data.sort_values(by='rating', inplace=True)

In [743]:
#sample = reviewTextProcessing['reviewText']
sample = "Bought one and loved it! Ordered another two.One is leaking right after installation!  a lot tries with dollars. Returned it!I thought it was just bad luck because review was so good.One week later, another one leaks when using it (does not leak when not using it).2 leaks out of three. I think it has something to do with QC.It would be 4~5 star if it does not leak...."
cleanUpTextParagraph(sample)

['buy one and love it',
 'order another two one be leak right after installation',
 'a lot try with dollar',
 'return it i think it be just bad luck because review be so good one week later another one leak when use it do not leak when not use it leak out of three',
 'i think it have something to do with qc it would be star if it do not leak']

In [ ]:
### Data clean so far